<a href="https://colab.research.google.com/github/Suryan5h/Natural-Language-Processing/blob/main/basics/wsd_lesk_algo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Word Sense Disambiguation

In [1]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [2]:
from nltk.corpus import wordnet as wn
from nltk import wsd

In [3]:
X = 'The die is cast.'
Y = 'Roll the die to get a 6.'
Z = 'What is dead may never die.'

In [4]:
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [5]:
wn.synsets('die')

[Synset('die.n.01'),
 Synset('die.n.02'),
 Synset('die.n.03'),
 Synset('die.v.01'),
 Synset('die.v.02'),
 Synset('die.v.03'),
 Synset('fail.v.04'),
 Synset('die.v.05'),
 Synset('die.v.06'),
 Synset('die.v.07'),
 Synset('die.v.08'),
 Synset('die.v.09'),
 Synset('die.v.10'),
 Synset('die.v.11')]

In [6]:
wn.synsets('die', pos=wn.NOUN)

[Synset('die.n.01'), Synset('die.n.02'), Synset('die.n.03')]

In [7]:
for syn in wn.synsets('die', pos=wn.NOUN):
    print(syn.definition())

a small cube with 1 to 6 spots on the six faces; used in gambling to generate random numbers
a device used for shaping metal
a cutting tool that is fitted into a diestock and used for cutting male (external) screw threads on screws or bolts or pipes or rods


In [8]:
for syn in wn.synsets('die', pos=wn.VERB):
    print(syn.definition())

pass from physical life and lose all bodily attributes and functions necessary to sustain life
suffer or face the pain of death
be brought to or as if to the point of death by an intense emotion such as embarrassment, amusement, or shame
stop operating or functioning
feel indifferent towards
languish as with love or desire
cut or shape with a die
to be on base at the end of an inning, of a player
lose sparkle or bouquet
disappear or come to an end
suffer spiritual death; be damned (in the religious sense)


Word-Sense Disambiguation with Lesk Algorithm

In [9]:
print(X)
wsd.lesk(X.split(), 'die')

The die is cast.


Synset('die.v.07')

In [10]:
_.definition()

'cut or shape with a die'

In [11]:
wsd.lesk(X.split(), 'die', pos=wn.NOUN).definition()

'a cutting tool that is fitted into a diestock and used for cutting male (external) screw threads on screws or bolts or pipes or rods'

In [12]:
print(Y)
wsd.lesk(Y.split(), 'die').definition()

Roll the die to get a 6.


'to be on base at the end of an inning, of a player'

In [13]:
wsd.lesk(Y.split(), 'die', pos=wn.NOUN).definition()

'a small cube with 1 to 6 spots on the six faces; used in gambling to generate random numbers'

In [14]:
print(Z)
wsd.lesk(Z.split(), 'die').definition()

What is dead may never die.


'a cutting tool that is fitted into a diestock and used for cutting male (external) screw threads on screws or bolts or pipes or rods'

In [15]:
wsd.lesk(Z.split(), 'die', pos=wn.VERB).definition()

'stop operating or functioning'

Automatic POS Tagging + Lesk with spaCy

In [16]:
!pip install spacy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [17]:
from spacy.cli import download
from spacy import load
# download('en_core_web_sm')
nlp = load('en_core_web_sm')

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [18]:
import warnings

POS_MAP = {
    'VERB': wn.VERB,
    'NOUN': wn.NOUN,
    'PROPN': wn.NOUN
}


def lesk(doc, word):
    found = False
    for token in doc:
        if token.text == word:
            word = token
            found = True
            break
    if not found:
        raise ValueError(f'Word \"{word}\" does not appear in the document: {doc.text}.')
    pos = POS_MAP.get(word.pos_, False)
    if not pos:
        warnings.warn(f'POS tag for {word.text} not found in wordnet. Falling back to default Lesk behaviour.')
    args = [c.text for c in doc], word.text
    kwargs = dict(pos=pos)
    return wsd.lesk(*args, **kwargs)

In [19]:
doc = nlp('Roll the die to get a 6.')

In [20]:
lesk(doc, 'die')

Synset('die.n.01')

In [21]:
lesk(doc, 'die').definition()

'a small cube with 1 to 6 spots on the six faces; used in gambling to generate random numbers'

In [22]:
lesk(nlp('I work at google.'), 'google').definition()

"a widely used search engine that uses text-matching techniques to find web pages that are important and relevant to a user's search"

In [23]:
lesk(nlp('I will google it.'), 'google').definition()

'search the internet (for information) using the Google search engine'

# The End